In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/evolve-academy/Evolve Academy - Technical Assessment - Vendor Data.xlsx
/kaggle/input/evolve-academy/Evolve Academy - Technical assessment - Invoice Transactions.csv
/kaggle/input/evolve-academy/Evolve Academy - Technical Assessment - Invoice Status.xlsx


**Load Data from CSV**

In [28]:
df = pd.read_csv('/kaggle/input/evolve-academy/Evolve Academy - Technical assessment - Invoice Transactions.csv')

**Task 1: Identify Duplicates**

In [36]:
from tabulate import tabulate

# Count the number of appearances for each invoice number
duplicated_invoices = df['Invoice Number'].value_counts().reset_index()

# Rename the columns for clarity
duplicated_invoices.columns = ['Invoice_Number', 'Count_of_Duplicates']

# Display the duplicated invoices using tabulate
print("\nDUPLICATED_INVOICES:\n")
print(tabulate(duplicated_invoices, headers='keys', tablefmt='fancy_grid', showindex=False))



DUPLICATED_INVOICES:

╒══════════════════╤═══════════════════════╕
│   Invoice_Number │   Count_of_Duplicates │
╞══════════════════╪═══════════════════════╡
│      2.02841e+08 │                     3 │
├──────────────────┼───────────────────────┤
│      1.04235e+08 │                     1 │
├──────────────────┼───────────────────────┤
│      1.70565e+08 │                     1 │
├──────────────────┼───────────────────────┤
│      2.27401e+08 │                     1 │
├──────────────────┼───────────────────────┤
│      1.96651e+08 │                     1 │
├──────────────────┼───────────────────────┤
│      2.19602e+08 │                     1 │
├──────────────────┼───────────────────────┤
│      1.49662e+08 │                     1 │
├──────────────────┼───────────────────────┤
│      1.74085e+08 │                     1 │
├──────────────────┼───────────────────────┤
│      1.48982e+08 │                     1 │
├──────────────────┼───────────────────────┤
│      1.2604e+08  │            

****

In [35]:
import pandas as pd
from tabulate import tabulate

# Rename the 'Vendor Number' column to 'Legacy Unique Identifier'
vendor_number = df.rename(columns={'Vendor Number': 'Legacy Unique Identifier'})

# Load the vendor data from the Excel file
vendor_data = pd.read_excel('/kaggle/input/assessment/Evolve Academy - Technical Assessment - Vendor Data.xlsx')

# Merge the two dataframes based on the common column 'Legacy Unique Identifier'
merged = df.join(vendor_data, how='left',  lsuffix="left", rsuffix="right")

# Select vendors with 'Invoice Status' as 'B' (Blocked)
invoice_status = merged[merged["Invoice Status"] == "B"]

# Extract the 'Name' column for blocked vendors
blocked_vendors = invoice_status[["Name"]].dropna().reset_index(drop=True)

# Print the list of blocked vendors using tabulate
print("\nBLOCKED_VENDORS:\n")
print(tabulate(blocked_vendors, headers=["Vendors_Name"], tablefmt='fancy_grid', showindex=False))



BLOCKED_VENDORS:

╒═══════════════════╕
│ Vendors_Name      │
╞═══════════════════╡
│ Coach Carter CC   │
├───────────────────┤
│ Yama Incorporated │
╘═══════════════════╛


**Task 3:**

In [40]:
from tabulate import tabulate

# Count the appearance of 'Incorporate Path' in the vendor data
vendor_count = vendor_data["Name"].value_counts()

# Reset the index to make it a DataFrame
vendor_table = vendor_count.reset_index()

# Extract the row for 'Incorporate Path'
incorporate_path = vendor_table.loc[4:4]

# Print the total count of 'Incorporate Path' vendors using tabulate
print("\nTOTAL_VENDOR:\n")
print(tabulate(incorporate_path.reset_index(), headers=["Name Of Vendor", "Total"], tablefmt='fancy_grid', showindex=False))



TOTAL_VENDOR:

╒════╤═══════════════════╤═════════╕
│    │ Name Of Vendor    │   Total │
╞════╪═══════════════════╪═════════╡
│  4 │ Incorporated Path │       1 │
╘════╧═══════════════════╧═════════╛


**Task 4:**

In [43]:
vendor_name = vendor_data['Name']

invoice_date = df["Invoice Date"]

date_format = pd.to_datetime(invoice_date, format='%d/%m/%Y %H:%M') #provide date format

year = date_format.dt.to_period('Y').reset_index() #extract year

rrc = merged.copy() #copy dataframe from a combined dataframe made earlier

rrc["RANK"] = df['Invoice Total Amount'].rank() #calculate rank 

rrc1 = rrc.join(year,lsuffix='left',rsuffix='right') #add the column to dataframe

rrc2 =rrc1.rename(columns= {'Invoice Dateright':'Year','Name': 'Vendor_Name'},inplace=False)

rrc3 = rrc2[['Vendor_Name','Year','Invoice Total Amount','RANK']]

print(tabulate(rrc3.dropna(),headers = ['Vendor_Name','Year','Invoice Total Amount','RANK'] ,tablefmt='fancy_grid',showindex = False))


╒═════════════════════════════════╤════════╤════════════════════════╤════════╕
│ Vendor_Name                     │ Year   │ Invoice Total Amount   │   RANK │
╞═════════════════════════════════╪════════╪════════════════════════╪════════╡
│ Killer of Sheep LTD             │ 2017   │ 7177509.94             │     23 │
├─────────────────────────────────┼────────┼────────────────────────┼────────┤
│ Black Sunday Engineering        │ 2016   │ 330,007.59             │     17 │
├─────────────────────────────────┼────────┼────────────────────────┼────────┤
│ A Simple Path Incorporated      │ 2017   │ 7757552.4              │     25 │
├─────────────────────────────────┼────────┼────────────────────────┼────────┤
│ Incorporated Path               │ 2017   │ 3742999.39             │     18 │
├─────────────────────────────────┼────────┼────────────────────────┼────────┤
│ All The King’S Men Incorporated │ 2017   │ 14724353.41            │      8 │
├─────────────────────────────────┼────────┼────────